In [1]:

# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from torch.mps import empty_cache
import torch
from langchain.chains import RetrievalQA


In [2]:
class GLM(LLM):
    max_token: int = 2048
    temperature: float = 0.8
    top_p = 0.9
    tokenizer: object = None
    model: object = None
    history_len: int = 1024
    
    def __init__(self):
        super().__init__()
        
    @property
    def _llm_type(self) -> str:
        return "GLM"
            
    def load_model(self, llm_device="gpu",model_name_or_path=None):
        model_config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name_or_path, config=model_config, trust_remote_code=True, device='cuda:3').half() # GLM模块装在gpu: 6



    def _call(self,prompt:str,history:List[str] = [],stop: Optional[List[str]] = None):
        response, _ = self.model.chat(
                    self.tokenizer,prompt,
                    history=history[-self.history_len:] if self.history_len > 0 else [],
                    max_length=self.max_token,temperature=self.temperature,
                    top_p=self.top_p)
        return response

In [3]:
import sys
modelpath = "/data1/dxw_data/llm/chatglm3-6b-128k"
sys.path.append(modelpath)
llm = GLM()
llm.load_model(model_name_or_path = modelpath)
#---------------------------至此, 成功加载模型

2024-06-17 09:44:01.418479: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 09:44:01.538067: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 09:44:02.092020: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2024-06-17 09:44:02.092086: W tensorflow/compiler/xla/stream_exec

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
# ----------------------------合并回答------------------------ #

In [14]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import json

# 定义PromptTemplate
prompt = PromptTemplate(
    input_variables=["caption", "comment"],
    template=(# "以下是一个元素的caption和comment，请分别概括说明它们的主题和内容，"
              "并参考以下主题之一：\n"
              "[商品导购, 商品评测, 穿搭推荐, 购物指南 ], [泛知识传播, 科普, 软件教程, 医疗健康, 留学备考, 科技 ], [剧情类, 电影解说, 影视剪辑, 情景短剧, 动漫剧集, 微纪录片 ], [休闲搞笑, 创意剪辑/配音, 吐槽恶搞, 动作表演, 风景摄影, 多米诺骨牌 ], [生活分享, 美食, vlog/感悟分享, 旅游, 健身美妆, 技能分享, 家居生活,] , [颜值类, 舞蹈, 音乐, cosplay ], [广告宣传, 地方文旅, 公益广告 ], [新闻时事, 政治/社会新闻, 热点剖析 ], [游戏, 体育, 汽车, 财经, 明星娱乐。]\n\n"
              "Caption: {caption}\n\nComment: {comment}\n\n"
              "先综合合并概括caption和comment元素的主题和内容。然后分析属于上述哪一类主题，进一步分析概括。")
)

# 创建LLMChain
# llm = OpenAI(api_key="YOUR_API_KEY")
chain = LLMChain(llm=llm, prompt=prompt)

# 读取merged_captions.json文件
with open('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/merged_captions.json', 'r', encoding='utf-8') as file:
    merged_data = json.load(file)

# 初始化保存结果的字典
summary_data = {}

# 生成每个元素的主题和内容概述
for i, (key, value) in enumerate(merged_data.items()):
    caption = value["caption"]
    comment = value["comment"]
    result = chain.run(caption=caption, comment=comment)
    summary_data[key] = result
    print(result)
    print(f"----------第{i}次完成----------")

# 输出结果到一个新的JSON文件
summary_data_file_path = "/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/summary_topic.json"
with open(summary_data_file_path, 'w', encoding='utf-8') as summary_file:
    json.dump(summary_data, summary_file, ensure_ascii=False, indent=4)

print(f"Summary JSON file saved to: {summary_data_file_path}")


Caption: 在香港迪士尼乐园的魔雪奇缘世界，一位蓝眼睛的女游客站在魔法桥上，背景是黑色，她穿着蓝色的裙子，场景非常梦幻。

Comment: 图中展示了香港迪士尼乐园的魔雪奇缘世界，游客在魔法桥上享受梦幻般的场景。同时，图中还出现了与冰雪奇缘相关的热门梗“烟花薯片”。

根据以上分析，该图片属于“旅游景点”主题，展示了香港迪士尼乐园的魔雪奇缘世界。
----------第0次完成----------
Caption和Comment元素的主题是电动牙刷的广告宣传。

内容概括：

Caption：这是一个抖音短视频的截图，展示了电动牙刷的广告宣传页面。

Comment：评论中提到了“就一‘刷’难求”，表明该电动牙刷非常受欢迎，供不应求。

根据上述内容分析，这个主题属于商品导购类，主要介绍了电动牙刷的广告宣传和评论，旨在推广和介绍产品。
----------第1次完成----------
根据caption和comment的内容，我们可以看出这是一张猫咪的短视频截图，背景是白色的，猫咪正在床上睡觉，床单是白色的。视频的评论区有留言，有点赞，播放量已经达到了一百万。从评论内容来看，有关于猫咪的话题，如“小奶猫”、“心态全写在脸上的猫猫”、“热搜：猫猫有多迷人”等。结合caption中的“图是一张猫咪的短视频截图”，我们可以判断这是一个猫咪相关的短视频。

从评论内容来看，大部分评论都是在讨论猫咪的可爱和迷人，还有一些评论是在分享猫咪的生活趣事。从视频的播放量来看，它已经吸引了大量的观众，达到了一百万。因此，根据主题和内容，我们可以判断这是一条关于猫咪的短视频，主题是萌宠和娱乐。
----------第2次完成----------
综合caption和comment元素，我们可以看到这是一幅关于一个身穿学生装的女生在直播的图片。背景是白色的，女生手上戴着一个蓝色的手表。评论中提到了《班级里的某些女43》这一剧情和校园主题，以及图片的4.6万浏览量和1.3万点赞量。根据这些信息，我们可以判断这幅图片属于“剧情类”主题，因为它涉及到一个校园剧情的直播画面。同时，图片的较高浏览量和点赞量也证明了这一主题的受欢迎程度。
----------第3次完成----------
Caption: 这是一张抖音视频截图，视频标题是“《班级里的某些女生43》#剧情#校园”，点赞数为1